# Data Processing Of MCMC Simulation Data from Binary Alloy

The main objective of this notebook is to document and illustrate all data analysis for determination of critical temperatures and graphics generation for my Theoretical Investigations Project. I have already determined mean field critical curves for the BEG model, and in this notebook I set out to:

1. Produce Magnetisation profiles for several values of vancy and atom chemical potential difference.
1. Compute critical temperatures for several Lattice sizes and vacancy - atom chemical potential difference.
1. Use finite size scaling to determine a set of critical points in $h - T$ phase space projection.

## Magnetisation Profiles fior Different Lattice Sizes and Chemical Potential Differences

Remember that the sampled Hamiltonian corresponds to a ferromagnetic BEG Model, on a centered rectangular lattice, with a non-zero quadrtupolar anisotropy term, in the presence of an staggered external magnetic field:

$$H = -\frac{J}{2} \sum\limits_{\langle i_{A_2}, j_{B_2}\rangle} S_{i_{A_2}} S_{j_{B_2}} + \mu\sum\limits_{i} S_i^2 + h\sum\limits_{i_B} S_{i_B} - h\sum\limits_{i_A} S_{i_A}$$

I used conventional MCMC (Markov Chain Monte Carlo) to sample the Gibbs-Boltzman distribution corresponding to this system. Just to remember, the system under study is a bipartite, substitutional, non-stoichiometric, binary alloy. I used MCMC techniques to compute equilibrium values of order parameter:

$$m = \langle S_{i_{A_2}} \rangle + \langle S_{i_{B_2}}\rangle$$

I know that magnetisation profile depends not only on temperature, but also on lattice size. Therefore, I computed several datasets corresponding to different values of $\mu$, $h$, temperature ($T$) and lattice size ($L$). I normalised unit energies so that $J = 1$, and units of entropy so that Boltzman's constant, $k_B = 1$.

Each dataset corresponds to magnetisation for 30 equaly spaced temperatures between $T_{\text{min}}=0.05$ and $T_{\text{max}} = 3.5$. I computed magnetisation profiles for values $\mu = 0.0, -1.7$ and $h = 0.0, 0.1, 0.3, 0.5$. Datasets correpsonds to lattice sizes $L = 8, 16, 32, 55$. The dataset files are saved as csv, and the names are **AltL_Mags_mu_h.csv**, in accordance to the following convention:

| Index | $L$ | $\mu$ | $h$ |
|:-----:|:---:|:-----:|:---:|
|  $0$  | $55$|  $0$  |$0.0$|
|  $1$  | $8$ | $-1.7$|$0.1$|
|  $2$  | $16$|   -   |$0.3$|
|  $3$  | $32$|   -   |$0.5$|
